### Uses Paper Trade - test money
##### https://alpaca.markets/docs/api-documentation/api-v2/

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import time, datetime
import pandas as pd

import alpha, database as db
import alpaca_trade as alpaca
from populate import *
from portfolios import *
from state import *

DataStore = db.DataStore()

In [2]:
alpaca.get_positions()

[]

In [3]:
state = State(DataStore, 'AAPL', indicators=['ADX', 'CCI', 'EMA'])

Available to use:
  <classname>.prices
  <classname>.indicators


In [20]:
import gym
import numpy as np
from gym import spaces
import pandas as pd

# class ConvergeCheck:
#     def __init__(self, wait=10):
#         '''wait is how long to wait after loss goes up'''
#         self.wait = wait
#         self.counter = 0
#         self.previous_loss = None
        
#     def divergence_check(self, loss):
#         if self.previous_loss is None:
#             self.previous_loss = loss
#         elif loss > self.previous_loss:
#             self.counter += 1
#             if self.counter >= self.wait:
#                 return True
#         else:
#             self.previous_loss = loss
#             self.counter = 0
#         return False
    
#     def convergence_check(self, trades)
    
#         if trades.equals(trades_dupl):
#         converged = True
#     else:
#         trades_dupl = trades.copy(deep = True)
#         trades.iloc[:,:] = 0

class Portfolio:
    def __init__(self):
        self.cash_remaining = int(alpaca.get_account()['cash'])
        self.positions = alpaca.get_positions()
        
    def position(self, sym):
        if sym in self.positions: # not sure format positions returned...
            alpaca.get_position(sym)
        else:
            print(f'no position in {sym}'')
        

class StockTrader(gym.Env):
    def __init__(self,
                portfolio=None,
                sym = None,
                dfPrices=None, 
                dfIndicators=None, 
                short=False):
        """dfIndicators is a DataFrame where each column is a different indicator; short=True would allow shorting position"""
        assert portfolio is not None, "need a portfolio"
        assert sym is not None, "what symbol is this?"
        assert indicators is not None and isinstance(indicators, pd.DataFrame), "need indicator dataframe"
        assert prices is not None and isinstance(prices, pd.DataFrame), "need prices dataframe"
        
        self.portfolio = portfolio
        self.indicators = dfIndicators
        self.prices = dfPrices
        num_indicators = self.indicators.shape[1]
        assert num_indicators > 0, "supply 1 or more indicators"

        self.action_space = spaces.Discrete(3)

        # set space for alpha indicators at +- infinity...?
        low_array = np.full((num_indicators), -np.inf)
        high_array = np.full((num_indicators), np.inf)
        self.observation_space = spaces.Box(low=low_array, high=high_array, dtype=np.float64)
        
        self.previous_price = 0  # didn't exist before first day, so set previous_price to 0
        first_day = self.indicators.index[0] # starting at first day indicators exist
        self.prices = self.prices.loc[first_day:] # rewriting prices to fit indicator list
        
        self.state = np.array(self.indicators.iloc[0]) # first day is inititial state
        self.days = iter(self.prices.index.values)
        
        # Iterate through days, checking action/reward, etc. in step...
        self.trades = pd.DataFrame(0, index = self.prices.index, columns = self.prices.columns)
        self.trades_dupl = self.trades.copy(deep = True)
        
        # position is how much long (positive), short (negative) or holding (zero)
        # NOTE: repalce with API to get position (add SYM to class to get symbol's position)
        self.position = 0  # portfolio.position(sym)  # how parse?

    def reset(self):
        self.previous_price = 0
        self.days = iter(self.prices.index.values)
        self.state = np.array(self.indicators.iloc[0])
        self.trades = pd.DataFrame(0, index = self.prices.index, columns = self.prices.columns)
        self.position = 0
    
    def step(self, action):
        #https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py
        err_msg = "%r (%s) invalid" % (action, type(action))
        assert self.action_space.contains(action), err_msg
        
        #Calculate reward here... first day = 0
        # 0 is buy, 1 is sell, 2 is hold
        try:
            new_day = next(trader.days)
            current_price = self.prices.loc[new_day]
            self.state = self.indicators.loc[new_day]

            if action == 0 and current_price > self.previous_price:
                reward = 2
            elif action == 0 and current_price < self.previous_price:
                reward = -2
            elif action == 1 and current_price < self.previous_price:
                reward = 2
            elif action == 1 and current_price > self.previous_price:
                reward = -2
            elif action == 2 and (current_price > self.previous_price or current_price < self.previous_price):
                reward = -2
            elif action == 2 and current_price == self.previous_price:
                reward = 2
            else:
                reward = 0
                
            if action == 0 and self.position == 0:
                
               # buying into a position

            self.previous_price = current_price
            done = False
        except StopIteration:
            if self.trades.equals(self.trades_dupl):
                done = True
            else:
                done = False
                self.trades_dupl = self.trades.copy(deep = True)

        return self.state, reward, done, {}

In [ ]:
sym = 'AAPL'
state = State(DataStore, sym, indicators=['ADX', 'CCI', 'EMA'])
StockTrader(state) # make it so can pass in state, which gets prices, values, the symbol, 

ToDo: refactor - state and portfolio combined? portfolio is holdings; state is a symbol's historic data
    perhaps State is renamed -> History?

In [21]:
trader = StockTrader(state.prices, state.indicators)

In [32]:
trader.step(1)
trader.reset()
trader.state

ADX    1.0
CCI    1.0
EMA    1.0
Name: 1999-11-29 00:00:00, dtype: float64